In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env

# Exercises
- file name: `time_series_anomaly_detection.py` or `time_series_anomaly_detection.ipynb`

The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.

**Hint**: You will need to explore the database (yay SQL!)

1. Label students by the program they are in.
1. Is it possible to identify the user_id of a staff member?
1. Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.
1. Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2. 
1. Plot the access activity of these students.

**BONUS:** Can you identify users who are viewing both the web dev and data science curriculum?

## Acquire

After doing some research, some experimentation of performing actions and watching the logs, we discovered what each of the fields represent. We then parse and name the fields accordingly.

In [2]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = '''
Select * from logs
left join cohorts on logs.cohort_id = cohorts.id
ORDER BY date ASC, time ASC;
'''
df = pd.read_sql(query, url)
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0


In [78]:
df.shape

(900223, 12)

## Prepare

- Need to concat date and time together
- drop id, updated_at, created_at, deleted_at
- map program id 1: data science 2: web dev
- convert date and time to date/time object, corhort id to int, start_date and end_date to time/time object
- set index to date time

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847330 non-null  float64
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       object 
 14  program_id  847330 non-null  float64
dtypes: float64(3), int64(1), object(11)
memory usage: 103.0+ MB


In [76]:
df

,date,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,program_id,timestamp
0,2018-01-26,2022-08-22 09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,2018-01-26 09:55:03-22:00
1,2018-01-26,2022-08-22 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,2018-01-26 09:56:02-22:00
2,2018-01-26,2022-08-22 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,2018-01-26 09:56:05-22:00
3,2018-01-26,2022-08-22 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,2018-01-26 09:56:06-22:00
4,2018-01-26,2022-08-22 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2.0,2018-01-26 09:56:24-22:00
...,...,...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,2022-08-22 16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,#,2014-02-04,2014-02-04,2.0,2021-04-21 16:41:51-22:00
900219,2021-04-21,2022-08-22 16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,#,2014-02-04,2014-02-04,2.0,2021-04-21 16:42:02-22:00
900220,2021-04-21,2022-08-22 16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,#,2014-02-04,2014-02-04,2.0,2021-04-21 16:42:09-22:00
900221,2021-04-21,2022-08-22 16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,Staff,#,2014-02-04,2014-02-04,2.0,2021-04-21 16:44:37-22:00


In [84]:
df.name.value_counts()

Staff         84031
Ceres         40730
Zion          38096
Jupiter       37109
Fortuna       36902
Voyageurs     35636
Ganymede      33844
Apex          33568
Deimos        32888
Darden        32015
Teddy         30926
Hyperion      29855
Betelgeuse    29356
Ulysses       28534
Europa        28033
Xanadu        27749
Bayes         26538
Wrangell      25586
Andromeda     25359
Kalypso       23691
Curie         21582
Yosemite      20743
Bash          17713
Luna          16623
Marco         16397
Easley        14715
Lassen         9587
Arches         8890
Florence       8562
Sequoia        7444
Neptune        7276
Olympic        4954
Kings          2845
Pinnacles      2158
Hampton        1712
Oberon         1672
Quincy         1237
Niagara         755
Mammoth         691
Glacier         598
Joshua          302
Ike             253
Badlands         93
Franklin         72
Apollo            5
Denali            4
Everglades        1
Name: name, dtype: int64

In [73]:
df[df['program_id'].isna()]

,date,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,program_id,timestamp
411,2018-01-26,2022-08-22 16:46:16,/,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 16:46:16-22:00
412,2018-01-26,2022-08-22 16:46:24,spring/extra-features/form-validation,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 16:46:24-22:00
425,2018-01-26,2022-08-22 17:54:24,/,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 17:54:24-22:00
435,2018-01-26,2022-08-22 18:32:03,/,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 18:32:03-22:00
436,2018-01-26,2022-08-22 18:32:17,mysql/relationships/joins,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 18:32:17-22:00
...,...,...,...,...,...,...,...,...,...,...,...,...
899897,2021-04-21,2022-08-22 12:49:00,javascript-ii,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:49:00-22:00
899898,2021-04-21,2022-08-22 12:49:02,javascript-ii/es6,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:49:02-22:00
899899,2021-04-21,2022-08-22 12:51:27,javascript-ii/map-filter-reduce,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:51:27-22:00
899900,2021-04-21,2022-08-22 12:52:37,javascript-ii/promises,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:52:37-22:00


In [81]:
df[df['cohort_id'].isna()]

,date,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,program_id,timestamp
411,2018-01-26,2022-08-22 16:46:16,/,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 16:46:16-22:00
412,2018-01-26,2022-08-22 16:46:24,spring/extra-features/form-validation,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 16:46:24-22:00
425,2018-01-26,2022-08-22 17:54:24,/,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 17:54:24-22:00
435,2018-01-26,2022-08-22 18:32:03,/,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 18:32:03-22:00
436,2018-01-26,2022-08-22 18:32:17,mysql/relationships/joins,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 18:32:17-22:00
...,...,...,...,...,...,...,...,...,...,...,...,...
899897,2021-04-21,2022-08-22 12:49:00,javascript-ii,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:49:00-22:00
899898,2021-04-21,2022-08-22 12:49:02,javascript-ii/es6,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:49:02-22:00
899899,2021-04-21,2022-08-22 12:51:27,javascript-ii/map-filter-reduce,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:51:27-22:00
899900,2021-04-21,2022-08-22 12:52:37,javascript-ii/promises,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:52:37-22:00


In [74]:
df[df['start_date'].isna()]

,date,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,program_id,timestamp
411,2018-01-26,2022-08-22 16:46:16,/,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 16:46:16-22:00
412,2018-01-26,2022-08-22 16:46:24,spring/extra-features/form-validation,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 16:46:24-22:00
425,2018-01-26,2022-08-22 17:54:24,/,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 17:54:24-22:00
435,2018-01-26,2022-08-22 18:32:03,/,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 18:32:03-22:00
436,2018-01-26,2022-08-22 18:32:17,mysql/relationships/joins,48,NaN,97.105.19.61,None,None,NaT,NaT,NaN,2018-01-26 18:32:17-22:00
...,...,...,...,...,...,...,...,...,...,...,...,...
899897,2021-04-21,2022-08-22 12:49:00,javascript-ii,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:49:00-22:00
899898,2021-04-21,2022-08-22 12:49:02,javascript-ii/es6,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:49:02-22:00
899899,2021-04-21,2022-08-22 12:51:27,javascript-ii/map-filter-reduce,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:51:27-22:00
899900,2021-04-21,2022-08-22 12:52:37,javascript-ii/promises,717,NaN,136.50.102.126,None,None,NaT,NaT,NaN,2021-04-21 12:52:37-22:00


In [83]:
df.loc[df['program_id'] == 2]

,date,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,program_id,timestamp
4,2018-01-26,2022-08-22 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2.0,2018-01-26 09:56:24-22:00
5,2018-01-26,2022-08-22 09:56:41,javascript-i/loops,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2.0,2018-01-26 09:56:41-22:00
6,2018-01-26,2022-08-22 09:56:46,javascript-i/conditionals,3,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2.0,2018-01-26 09:56:46-22:00
7,2018-01-26,2022-08-22 09:56:48,javascript-i/functions,3,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2.0,2018-01-26 09:56:48-22:00
8,2018-01-26,2022-08-22 09:56:59,javascript-i/loops,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2.0,2018-01-26 09:56:59-22:00
...,...,...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,2022-08-22 16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,#,2014-02-04,2014-02-04,2.0,2021-04-21 16:41:51-22:00
900219,2021-04-21,2022-08-22 16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,#,2014-02-04,2014-02-04,2.0,2021-04-21 16:42:02-22:00
900220,2021-04-21,2022-08-22 16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,#,2014-02-04,2014-02-04,2.0,2021-04-21 16:42:09-22:00
900221,2021-04-21,2022-08-22 16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,Staff,#,2014-02-04,2014-02-04,2.0,2021-04-21 16:44:37-22:00


In [60]:
# Convert Year into Integer

df.date = pd.to_datetime(df.date)
df.time = pd.to_datetime(df.time)
df.start_date = pd.to_datetime(df.start_date)
df.end_date = pd.to_datetime(df.end_date)
df['timestamp']=df.apply(lambda x:'%s-%s' % (x['date'],x['time']),axis=1)
df.timestamp = pd.to_datetime(df.timestamp)
df = df.drop(columns=['updated_at','created_at', 'id', 'deleted_at'])
df['cohort_id'] = df.cohort_id.astype(int)
#df['program_id'] = df.program_id.map({1: 'data_science', 2: 'web_dev'})
df.dtypes

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)
pages = df['endpoint'].resample('d').count()
pages.head()